In [ ]:
#Preparing the Dataset
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

def extract_features(directory):
    features, labels = [], []
    for folder in os.listdir(directory):
        speaker_folder = os.path.join(directory, folder)
        if os.path.isdir(speaker_folder):
            for filename in os.listdir(speaker_folder):
                file_path = os.path.join(speaker_folder, filename)
                audio, sr = librosa.load(file_path, sr=None)
                mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
                if mfccs.shape[1] > 1:  # Ensure the audio is long enough for processing
                    mfccs = np.mean(mfccs.T, axis=0)  # Taking the mean of MFCCs over time
                    features.append(mfccs)
                    labels.append(folder)
    return np.array(features), np.array(labels)

# Load dataset
directory = '/content/drive/MyDrive/voice_sample'
features, labels = extract_features(directory)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels_categorical, test_size=0.2, random_state=42)


<ipython-input-3-15551f782f66>:16: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
#building the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

def build_model(input_shape, num_classes):
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=input_shape),
        LSTM(128),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #categorical_crossentropy-> used for multi-classes classification
    return model

# Define model
model = build_model((X_train.shape[1], 1), y_train.shape[1])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
#training model
## Reshape for LSTM layer
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.1613 - loss: 1.6284 - val_accuracy: 0.3750 - val_loss: 1.5379
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.2581 - loss: 1.5870 - val_accuracy: 0.2500 - val_loss: 1.4847
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - accuracy: 0.2581 - loss: 1.5670 - val_accuracy: 0.3750 - val_loss: 1.4355
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.3226 - loss: 1.4549 - val_accuracy: 0.5000 - val_loss: 1.3930
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.4194 - loss: 1.4610 - val_accuracy: 0.5000 - val_loss: 1.3479
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - accuracy: 0.4516 - loss: 1.4616 - val_accuracy: 0.5000 - val_loss: 1.2997
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - accuracy: 0.3871 - loss: 1.4187 - val_accuracy: 0.5000 - val_loss: 1.2721
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - accuracy: 0.4516 - loss: 1.3151 - val_accuracy: 0.5000 - val_loss: 1.

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

# Save model
model.save('speaker_identification_model.h5')

1/1 [==============================] - 0s 41ms/step - loss: 1.0810 - accuracy: 0.6250
Test accuracy: 62.50%


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def predict_speaker(file_path, model, label_encoder):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    mfccs_reshaped = mfccs_mean[np.newaxis, ..., np.newaxis]
    prediction = model.predict(mfccs_reshaped)
    speaker = label_encoder.inverse_transform([np.argmax(prediction)])
    return speaker[0]

# Example of usage
speaker = predict_speaker('/content/drive/MyDrive/Recording.m4a', model, label_encoder)
print("Predicted Speaker:", speaker)


<ipython-input-10-152f64d46224>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted Speaker: Rishabh


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
